In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import sys
import altair as alt
sys.path.append('../src')
from models import plot_structure, plot_supply_and_demand_canada, regression_discontinuity_model, differences_differences, plot_for_parallel_trends 

#categories for products and services gotten by LLMs
df_model_data_CAN = pd.read_csv('../data/processed/CAN_Categorized_Products_and_Services.csv')
df_model_data_USA = pd.read_csv('../data/processed/US_Categorized_Products_and_Services.csv')

#CPI data for Canada and USA to analyze grocery and clothing and footwear inflation
df_Canada_CPI = pd.read_csv('../data/processed/Canada_CPI_Processed_2018_2019.csv')
df_USA_CPI = pd.read_csv('../data/processed/USA_CPI_Processed_2018_2019.csv')

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

#standardize and get scaled and non-scaled data for CPI data
df_USA_CPI_melt = pd.melt(df_USA_CPI, var_name = 'Products and product groups',value_name = 'VALUE', id_vars = 'REF_DATE')

df_Canada_CPI_Scaled = df_Canada_CPI.copy()
df_Canada_CPI_NonScaled = df_Canada_CPI.copy()

df_USA_CPI_Scaled = df_USA_CPI_melt.copy()
df_USA_CPI_NonScaled = df_USA_CPI_melt.copy()
for unique_product in df_Canada_CPI_Scaled['Products and product groups'].unique():
    stan_scale = StandardScaler()
    mask = df_Canada_CPI_Scaled['Products and product groups']==unique_product
    df_Canada_CPI_Scaled.loc[mask, "VALUE"] = stan_scale.fit_transform(df_Canada_CPI_Scaled.loc[mask, "VALUE"].values[:, None]).flatten()
for unique_product in df_USA_CPI_Scaled['Products and product groups'].unique():
    stan_scale = StandardScaler()
    mask = df_USA_CPI_Scaled['Products and product groups']==unique_product
    df_USA_CPI_Scaled.loc[mask, "VALUE"] = stan_scale.fit_transform(df_USA_CPI_Scaled.loc[mask, "VALUE"].values[:, None]).flatten()

In [3]:
#Canada and US datasets scaled and non-scaled and make category columns
dict_CAN = df_model_data_CAN.drop_duplicates(subset=['Product_Service']).set_index('Product_Service')['Category'].to_dict()
dict_USA = df_model_data_USA.drop_duplicates(subset=['Product_Service']).set_index('Product_Service')['Category'].to_dict()

df_Canada_CPI_Scaled['Category'] = df_Canada_CPI['Products and product groups'].map(dict_CAN)
df_Canada_CPI_NonScaled['Category'] = df_Canada_CPI['Products and product groups'].map(dict_CAN)
df_USA_CPI_Scaled['Category'] = df_USA_CPI_Scaled['Products and product groups'].map(dict_USA)
df_USA_CPI_NonScaled['Category'] = df_USA_CPI_NonScaled['Products and product groups'].map(dict_USA)


In [4]:
#make groupby dataframe get median of each category
df_Canada_CPI_Cat = df_Canada_CPI_Scaled.copy().drop(columns = ['Products and product groups'])
df_Canada_CPI_groupby = df_Canada_CPI_Cat.groupby(['REF_DATE', 'Category']).median().reset_index()

df_Canada_CPI_Cat_NonScaled = df_Canada_CPI_NonScaled.copy().drop(columns = ['Products and product groups'])
df_Canada_CPI_groupby_NonScaled = df_Canada_CPI_Cat_NonScaled.groupby(['REF_DATE', 'Category']).median().reset_index()

df_USA_CPI_Cat = df_USA_CPI_Scaled.copy().drop(columns = ['Products and product groups'])
df_USA_CPI_groupby = df_USA_CPI_Cat.groupby(['REF_DATE', 'Category']).median().reset_index()

df_USA_CPI_Cat_NonScaled = df_USA_CPI_NonScaled.copy().drop(columns = ['Products and product groups'])
df_USA_CPI_groupby_NonScaled = df_USA_CPI_Cat_NonScaled.groupby(['REF_DATE', 'Category']).median().reset_index()

In [5]:
#look at all potential categories
for column in df_Canada_CPI_groupby['Category'].unique():
    print(column)

???
Alcohol & Tobacco
Appliances & Equipment
Clothing & Footwear
Communication
Education & Reading
Energy
Furniture & Home Decor
Gardening
Groceries
Health & Personal Care
Household Supplies
Housing & Utilities
Housing, Rent, Tax  & Insurance
Recreation & Entertainment
Services
Transportation
Utilities
Vehicles


In [ ]:
#look at plot groceries which was more impacted and compare to housing, rent, tax and insurance which was less impacted by tariffs
plot_for_parallel_trends(df_Canada_CPI_groupby, '2018-01-01', '2018-11-01', 'Groceries', 'Housing, Rent, Tax  & Insurance')

/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/models/rc_differences.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [7]:
#look at china tariffs for differences in differences to get a better idea for one month
differences_differences(df_Canada_CPI_groupby, 'Groceries', 'Housing, Rent, Tax  & Insurance', '2018-07-01', '2018-09-01', '2018-08-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     15.16
Date:                Thu, 20 Mar 2025   Prob (F-statistic):             0.0625
Time:                        18:56:28   Log-Likelihood:               -0.55065
No. Observations:                   6   AIC:                             9.101
Df Residuals:                       2   BIC:                             8.268
Df Model:                           3                                         
Covariance Type:                  HC3                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.1713      0.590      0.290      0.772      -0.985       1.327
tariff_non_tariffed         -0.3207      0.650     -0.494      0.622      -1.594       0.953
post                         1.4613      1.021      1.431      0.152      -0.540       3.462
tariff_non_tariffed:post    -2.9879      1.064     -2.808      0.005      -5.073      -0.902
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.531
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.189
Skew:                           0.000   Prob(JB):                        0.910
Kurtosis:                       2.130   Cond. No.                         6.32
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [8]:
#It seems like the groupby shows some interaction 
#let us investigate into specific grocery categories to see which groceries may have been impacted by the tariffs

In [9]:

grocery_items = df_Canada_CPI_NonScaled[(df_Canada_CPI_NonScaled['Category'] == 'Groceries')]['Products and product groups'].unique()

grocery_dataframe = df_Canada_CPI_NonScaled[(df_Canada_CPI_NonScaled['Category'] == 'Groceries') | (df_Canada_CPI_NonScaled['Category'] == 'Housing, Rent, Tax  & Insurance')]


dict_grocery = food_categories = {
    'Apples': 'Fruit',
    'Baby foods': 'Prepared Foods',
    'Bakery and cereal products (excluding baby food)': 'Bakery and Cereal',
    'Bakery products': 'Bakery and Cereal',
    'Bananas': 'Fruit',
    'Bread, rolls and buns': 'Bakery and Cereal',
    'Breakfast cereal and other cereal products (excluding baby food)': 'Bakery and Cereal',
    'Butter': 'Dairy',
    'Canned and other preserved fish': 'Fish and Seafood',
    'Canned vegetables and other vegetable preparations': 'Vegetables',
    'Cereal products (excluding baby food)': 'Bakery and Cereal',
    'Cheese': 'Dairy',
    'Coffee': 'Beverages',
    'Coffee and tea': 'Beverages',
    'Condiments, spices and vinegars': 'Condiments and Spices',
    'Confectionery': 'Sweets',
    'Cookies and crackers': 'Bakery and Cereal',
    'Dairy products': 'Dairy',
    'Dairy products and eggs': 'Dairy',
    'Edible fats and oils': 'Fats and Oils',
    'Eggs': 'Dairy',
    'Fish': 'Fish and Seafood',
    'Fish, seafood and other marine products': 'Fish and Seafood',
    'Flour and flour-based mixes': 'Bakery and Cereal',
    'Food': 'General Category',
    'Food and energy': 'General Category',
    'Food purchased from cafeterias and other restaurants': 'Dining Out',
    'Food purchased from fast food and take-out restaurants': 'Dining Out',
    'Food purchased from restaurants': 'Dining Out',
    'Food purchased from stores': 'General Category',
    'Food purchased from table-service restaurants': 'Dining Out',
    'Fresh fruit': 'Fruit',
    'Fresh fruit and vegetables': 'Produce',
    'Fresh milk': 'Dairy',
    'Fresh or frozen beef': 'Meat',
    'Fresh or frozen chicken': 'Meat',
    'Fresh or frozen fish (including portions and fish sticks)': 'Fish and Seafood',
    'Fresh or frozen meat (excluding poultry)': 'Meat',
    'Fresh or frozen pork': 'Meat',
    'Fresh or frozen poultry': 'Meat',
    'Fresh vegetables': 'Vegetables',
    'Frozen and dried vegetables': 'Vegetables',
    'Frozen food preparations': 'Prepared Foods',
    'Fruit juices': 'Beverages',
    'Fruit, fruit preparations and nuts': 'Fruit',
    'Goods excluding food purchased from stores': 'General Category',
    'Goods excluding food purchased from stores and energy': 'General Category',
    'Ham and bacon': 'Meat',
    'Ice cream and related products': 'Dairy',
    'Lettuce': 'Vegetables',
    'Margarine': 'Fats and Oils',
    'Meat': 'Meat',
    'Non-alcoholic beverages': 'Beverages',
    'Non-durable goods excluding food purchased from stores': 'General Category',
    'Non-durable goods excluding food purchased from stores and energy': 'General Category',
    'Nuts and seeds': 'Nuts and Seeds',
    'Oranges': 'Fruit',
    'Pasta products': 'Pasta',
    'Pet food and supplies': 'Pet Supplies',
    'Potatoes': 'Vegetables',
    'Preserved fruit and fruit preparations': 'Fruit',
    'Preserved vegetables and vegetable preparations': 'Vegetables',
    'Processed meat': 'Meat',
    'Rice and rice-based mixes': 'Grains',
    'Seafood and other marine products': 'Fish and Seafood',
    'Soup': 'Prepared Foods',
    'Sugar and confectionery': 'Sweets',
    'Sugar and syrup': 'Sweets',
    'Tea': 'Beverages',
    'Tomatoes': 'Vegetables',
    'Vegetables and vegetable preparations': 'Vegetables',
    'Homeowners': 'Housing, Rent, Tax  & Insurance',
    'Homeowners\' replacement cost': 'Housing, Rent, Tax  & Insurance',
    'Homeowners\' maintenance and repairs': 'Housing, Rent, Tax  & Insurance',
    'Mortgage interest cost': 'Housing, Rent, Tax  & Insurance',
    'Property taxes': 'Housing, Rent, Tax  & Insurance',
    'Rent': 'Housing, Rent, Tax  & Insurance',
    'Rented accommodation': 'Housing, Rent, Tax  & Insurance',
    'Shelter': 'Housing, Rent, Tax  & Insurance',
    'Shelter (1986)': 'Housing, Rent, Tax  & Insurance',
    'Tenants\' maintenance, repairs and other expenses': 'Housing, Rent, Tax  & Insurance'

}



In [10]:
#make grocery categories within the grocery category dataframe
grocery_dataframe['Category'] = grocery_dataframe['Products and product groups'].map(dict_grocery)

grocery_dataframe.drop(columns = ['Products and product groups'], inplace=True)

grocery_dataframe_groupby = grocery_dataframe.groupby(['REF_DATE', 'Category']).mean().reset_index()

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_58411/3400313063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grocery_dataframe['Category'] = grocery_dataframe['Products and product groups'].map(dict_grocery)
/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_58411/3400313063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grocery_dataframe.drop(columns = ['Products and product groups'], inplace=True)


In [11]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Fish and Seafood', 'Dairy')

#Dairy seems unimpacted due to the protection tariffs by Canada while fish and seafood the counterfactual seems to go down following the treatment period as a sort of recovery period

/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/models/rc_differences.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [12]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Grains', 'Pasta', 'Vegetables')

/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/models/rc_differences.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [13]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Sweets', 'Fruit')

/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/models/rc_differences.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [14]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Meat', 'Pet Supplies')

/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/models/rc_differences.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [15]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'General Category', 'Dining Out')

/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/models/rc_differences.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [16]:
#It seems most items follow a sharp downward trend in august when china tariffs were implemented and were not as affected by Canadian tariffs
#Only dairy and seafood and fish seem to follow the parallel trends assumption 
#let us investigate differences in differences to get an idea of the impact of the tariffs on groceries that followed the aforementioned pattern
#these groceries seem to have to do with everyday food group except carbohydrates and vegetables
differences_differences(grocery_dataframe_groupby, 'Fish and Seafood', 'Dairy', '2018-07-01', '2018-09-01', '2018-08-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.855
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     6.601
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.134
Time:                        18:56:28   Log-Likelihood:                -2.7231
No. Observations:                   6   AIC:                             13.45
Df Residuals:                       2   BIC:                             12.61
Df Model:                           3                                         
Covariance Type:                  HC3                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.3060      0.558     -0.548      0.584      -1.401       0.789
tariff_non_tariffed          0.7692      0.933      0.824      0.410      -1.060       2.598
post                         1.6425      1.145      1.434      0.152      -0.602       3.887
tariff_non_tariffed:post    -3.7570      1.378     -2.727      0.006      -6.458      -1.056
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.341
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.476
Skew:                          -0.000   Prob(JB):                        0.788
Kurtosis:                       1.621   Cond. No.                         6.32
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [17]:
#It seems the groupby shows a significant difference but when looking at the individual items, the difference is not significant
#perhaps there was not enough data and the set was underpowered but it provides some evidence from the plots that the tariffs may have had a slight impact on goods - let us investigate further

In [18]:
#df_sales for Canada and USA
df_sales = pd.read_csv("../data/processed/Canada_Sales_Processed.csv")

In [19]:
df_sales.columns = ['PrincipleStats', 'GoodType', 'REF_DATE', 'VALUE']
df_sales['PrincipleStats'].unique()

array(['Finished goods manufactured, estimated values at end of  month',
       'Goods or work in process, estimated values at end of month',
       'New orders, estimated values of orders received during month',
       'Raw materials, fuel, supplies, components, estimated values at end of month',
       'Sales of goods manufactured (shipments)',
       'Total inventory, estimated values of total inventory at end of the month',
       'Unfilled orders, estimated values of orders at end of month'],
      dtype=object)

In [75]:
for column in df_sales['GoodType'].unique():
    print(column)

Aerospace product and parts manufacturing [3364]
Apparel manufacturing [315]
Beverage and tobacco product manufacturing [312]
Chemical manufacturing [325]
Chocolate and chocolate confectionery manufacturing [31135]
Computer and electronic product manufacturing [334]
Durable goods industries
Electrical equipment, appliance and component manufacturing [335]
Fabricated metal product manufacturing [332]
Flour mixes, dough, and pasta manufacturing from purchased flour [311824]
Food manufacturing [311]
Furniture and related product manufacturing [337]
Industrial machinery manufacturing [33324]
Infants' cut and sew clothing manufacturing [315241]
Leather and allied product manufacturing [316]
Machinery manufacturing [333]
Manufacturing [31-33]
Miscellaneous manufacturing [339]
Motor vehicle body and trailer manufacturing [3362]
Motor vehicle manufacturing [3361]
Motor vehicle parts manufacturing [3363]
Non-durable goods industries
Non-metallic mineral product manufacturing [327]
Other cut and

In [20]:
#On august the unfilled orders was lowered meaning less food manufacturing demand and less supply to retail store and more inflation for that month
plot_supply_and_demand_canada(df_sales, "Food manufacturing [311]")

6524    7613953.0
6525    7619427.0
6526    7640012.0
6527    7686838.0
6528    7672824.0
Name: VALUE, dtype: float64


alt.Chart(...)

In [21]:
#it seems more inventory meant that less orders were filled - this could be due to the tariffs - and it also seems that there were fewer unfilled orders meaning less demand for food manufacturing which means supply went down and inflation should go up.
#but since demand and inventory are similar, it is hard to say if the tariffs had an impact on inflation and the original models without the groupby may be right 

In [22]:
#lets look earlier
plot_supply_and_demand_canada(df_sales, "Food manufacturing [311]", '2018-01-01', '2018-06-01')

6524    7613953.0
6525    7619427.0
6526    7640012.0
6527    7686838.0
6528    7672824.0
Name: VALUE, dtype: float64


alt.Chart(...)

In [23]:
model_Canada_RC_groceries_first_tariff, _, _, _ = regression_discontinuity_model(df_Canada_CPI_groupby, 'Groceries', '2017-10-01', '2018-06-01', '2018-03-01', '2018-04-01')

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     3.444
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.101
Time:                        18:56:28   Log-Likelihood:               -0.20393
No. Observations:                   9   AIC:                             6.408
Df Residuals:                       6   BIC:                             7.000
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6366      0.231      2.

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)


In [24]:

model_Canada_RC_groceries_first_tariff.summary()

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                 -0.184
Method:                 Least Squares   F-statistic:                    0.3956
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.762
Time:                        18:56:28   Log-Likelihood:                 1.0298
No. Observations:                   9   AIC:                             5.940
Df Residuals:                       5   BIC:                             6.729
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.1418      0.320     -0.444      0.657      -0.768       0.485
PredTreatment              0.3763      0.568      0.662      0.508      -0.738       1.490
Num_Date                  -0.0086      0.090     -0.095      0.924      -0.185       0.167
PredTreatment:Num_Date     0.1141      0.212      0.538      0.591      -0.302       0.530
==============================================================================
Omnibus:                        0.122   Durbin-Watson:                   2.519
Prob(Omnibus):                  0.941   Jarque-Bera (JB):                0.323
Skew:                          -0.133   Prob(JB):                        0.851
Kurtosis:                       2.112   Cond. No.                         9.53
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [25]:
model_Canada_RC_groceries_second_tariff, _, _, _ = regression_discontinuity_model(df_Canada_CPI_groupby, 'Groceries', '2018-05-01', '2018-12-01', '2018-07-01', '2018-10-01')

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                     8.231
Date:                Thu, 20 Mar 2025   Prob (F-statistic):             0.0262
Time:                        18:56:28   Log-Likelihood:                0.20998
No. Observations:                   8   AIC:                             5.580
Df Residuals:                       5   BIC:                             5.818
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5556      0.161      3.

In [26]:
model_Canada_RC_groceries_second_tariff.summary()

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.175
Method:                 Least Squares   F-statistic:                    0.6311
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.633
Time:                        18:56:28   Log-Likelihood:                 2.1432
No. Observations:                   8   AIC:                             3.714
Df Residuals:                       4   BIC:                             4.031
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.2901      0.824     -0.352      0.725      -1.906       1.326
PredTreatment              0.6103      1.546      0.395      0.693      -2.420       3.641
Num_Date                   0.1049      0.189      0.554      0.580      -0.266       0.476
PredTreatment:Num_Date    -0.3065      0.565     -0.543      0.587      -1.413       0.800
==============================================================================
Omnibus:                        0.207   Durbin-Watson:                   2.599
Prob(Omnibus):                  0.902   Jarque-Bera (JB):                0.198
Skew:                           0.229   Prob(JB):                        0.906
Kurtosis:                       2.380   Cond. No.                         19.4
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [27]:
first_tariff_vs_second_tariff, _, _, _ = regression_discontinuity_model(df_Canada_CPI_groupby, 'Groceries', '2017-10-01', '2018-12-01', '2018-05-01')

first_tariff_vs_second_tariff.summary()

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                 -0.192
Method:                 Least Squares   F-statistic:                   0.09946
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.959
Time:                        18:56:28   Log-Likelihood:                 1.2596
No. Observations:                  15   AIC:                             5.481
Df Residuals:                      11   BIC:                             8.313
Df Model:                           3                                         
Covariance Type:                  HC3                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -0.1487      0.365     -0.408      0.684      -0.864       0.566
above_or_below              0.1979      0.444      0.446      0.656      -0.672       1.067
Num_Date                   -0.0218      0.074     -0.296      0.767      -0.166       0.122
above_or_below:Num_Date     0.0132      0.094      0.141      0.888      -0.170       0.197
==============================================================================
Omnibus:                        0.180   Durbin-Watson:                   2.570
Prob(Omnibus):                  0.914   Jarque-Bera (JB):                0.106
Skew:                          -0.138   Prob(JB):                        0.948
Kurtosis:                       2.693   Cond. No.                         22.2
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [28]:
#it seems like american tariffs had little immediate impact on Canadian grocery retailers when looking at manufacturing even the only month affected was the China tariffs in August as mentioned supply was still high - it is hard to disentangle the impact of the tariffs and the news so perhaps we should look for a more long term approach to gain more insight to what happened with groceries


Canada_parallel_trends_groceries = plot_for_parallel_trends(df_Canada_CPI_groupby, '2017-01-01', '2019-10-01', 'Groceries', 'Education & Reading')



/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/models/rc_differences.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


In [29]:
#it is hard to quantify the impact of groceries based on specific tariffs due to the serial correlation of tariffs but looking from the start of the breakdown in China talks to the Chinese hike tax we will look at RD between these periods and see if we can draw conclusions
#there does seem to be greater variance from the start of the Intellectual Property Trials to the Chinese hike in tariffs that started propagating like a wave with each tariff with minimal activity following the results of the IP investigation - let us investigate further
model_groceries_longterm_IP, _, _, _ = regression_discontinuity_model(df_Canada_CPI_groupby, 'Groceries', '2017-01-01', '2018-04-01', '2017-08-01', heteroskedasticity = 'nonrobust')

model_groceries_longterm_IP.summary()

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     3.043
Date:                Thu, 20 Mar 2025   Prob (F-statistic):             0.0744
Time:                        18:56:28   Log-Likelihood:                 7.0097
No. Observations:                  15   AIC:                            -6.019
Df Residuals:                      11   BIC:                            -3.187
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.0225      0.121      0.187      0.855      -0.243       0.288
above_or_below             -0.3867      0.183     -2.110      0.059      -0.790       0.017
Num_Date                   -0.0218      0.033     -0.651      0.528      -0.095       0.052
above_or_below:Num_Date     0.0883      0.043      2.045      0.066      -0.007       0.183
==============================================================================
Omnibus:                        1.916   Durbin-Watson:                   2.464
Prob(Omnibus):                  0.384   Jarque-Bera (JB):                1.189
Skew:                          -0.407   Prob(JB):                        0.552
Kurtosis:                       1.887   Cond. No.                         25.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
#Next lets compare with RD the tariff / IP trails period to the Chinese hike in tariffs
model_groceries_longterm_recovery, _, _, _ = regression_discontinuity_model(df_Canada_CPI_groupby, 'Groceries', '2018-02-01', '2019-05-01', '2018-11-01', '2019-02-01', heteroskedasticity='nonrobust')

model_groceries_longterm_recovery.summary()

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     12.84
Date:                Thu, 20 Mar 2025   Prob (F-statistic):           0.000836
Time:                        18:56:28   Log-Likelihood:               -0.58892
No. Observations:                  16   AIC:                             7.178
Df Residuals:                      13   BIC:                             9.496
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6012      0.099      6.

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  res = hypotest_fun_out(*samples, **kwds)
/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.9014
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.469
Time:                        18:56:28   Log-Likelihood:               -0.13639
No. Observations:                  16   AIC:                             8.273
Df Residuals:                      12   BIC:                             11.36
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0829      0.102     -0.816      0.430      -0.304       0.138
PredTreatment              0.2419      0.227      1.065      0.308      -0.253       0.737
Num_Date                   0.0013      0.017      0.077      0.940      -0.036       0.039
PredTreatment:Num_Date     0.0600      0.064      0.942      0.365      -0.079       0.199
==============================================================================
Omnibus:                        0.669   Durbin-Watson:                   2.889
Prob(Omnibus):                  0.716   Jarque-Bera (JB):                0.663
Skew:                          -0.232   Prob(JB):                        0.718
Kurtosis:                       2.117   Cond. No.                         16.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
#The IP trials overall impact including the US tariffs on Canada shows a reduction on inflation in groceries in Canada while the Chinese hike in tariffs to the end of the data does not show a significant impact on groceries with the groupby

#the smaller impact of each tariff appear to cause a wave that ripples through time causing greater variance in the data, each individual month of tariffs increased inflation at the treatment period but appeared to reduce inflation the following month before another wave occurs. 

#overall groceries were not as impacted when grouped together as there were some groceries influenced such as seafood while other groceries such as dairy remained unimpacted 


#In other words the tariffs had a short term impact on inflation but contributed to larger ripples in waves that caused greater variance in the data.

#The news of breakdown in China talks momentarily caused economic shock that influenced the agricultural industry to decrease food supply in Canada and increase inflation
# and the proceeding recovery after the break down in China talks and beginning of IP trade investigations, did appear to make grocery inflation lower for a short period of time.





In [74]:

#let us plot the char for sales for food manufucturing during this time period again for further investigate invesigation

Canada_groceries_supply_demand = plot_supply_and_demand_canada(df_sales, "Food manufacturing [311]", "2017-02-01", "2019-12-01")

Canada_groceries_supply_demand

6524    7613953.0
6525    7619427.0
6526    7640012.0
6527    7686838.0
6528    7672824.0
Name: VALUE, dtype: float64


alt.Chart(...)

In [33]:
#It appears that demand for food manufacturing was lower than supply during the trade talk breakdown which means supply for food was limited during the trade talks which caused inflation but demand rose following the trade talk breakdown and remained unaffected throughout the tariffs which is why groceries were largely unaffected



In [34]:
#while inventory did increase meaning a decrease in supply, there was still enough demand and that is why groceries were not impacted despite the sharp decrease in August 2018 perhaps mainly due to the China tariffs on US goods.



In [35]:
#let us look at what happened in American groceries due to the tariffs and compare the effect of the tariffs on groceries for US tariffs vs Canadian tariffs
#they appear to have a similar effect on American goods

plot_structure(df_USA_CPI_groupby, 'Groceries', '2017-01-01', '2020-02-01')



Product:  Groceries


alt.Chart(...)

In [36]:
#It appears that the American grocery dataset follows the Canadian with waves except the breakdown in china talks led to a decrease in inflation for Canadian groceries but an increase for American groceries
# We will look at the IP investigation later but it appears trumps tariffs on Canadian goods had only a small impact on American groceries but the combination of Canadian announcement of tariffs, canadian tariffs and chinese tariffs on American exported goods may have had a larger impact, let us investigate further
model_USA_RC_grocery_cutoff_first_tariff, _, _, _ = regression_discontinuity_model(df_USA_CPI_groupby, 'Groceries', '2017-10-01', '2018-06-01', '2018-03-01', '2018-04-01')

print(model_USA_RC_grocery_cutoff_first_tariff.summary())
model_USA_RC_grocery_cutoff_second_tariff, _, _, _= regression_discontinuity_model(df_USA_CPI_groupby, 'Groceries', '2018-05-01', '2018-12-01', '2018-07-01', '2018-10-01')

model_USA_RC_grocery_cutoff_second_tariff.summary()

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     3.444
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.101
Time:                        18:56:28   Log-Likelihood:               -0.20393
No. Observations:                   9   AIC:                             6.408
Df Residuals:                       6   BIC:                             7.000
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6366      0.231      2.

...

alt.Chart(...)

alt.Chart(...)

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)
/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.689
Method:                 Least Squares   F-statistic:                     8.231
Date:                Thu, 20 Mar 2025   Prob (F-statistic):             0.0262
Time:                        18:56:28   Log-Likelihood:                0.20998
No. Observations:                   8   AIC:                             5.580
Df Residuals:                       5   BIC:                             5.818
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5556      0.161      3.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.905
Method:                 Least Squares   F-statistic:                     4.498
Date:                Thu, 20 Mar 2025   Prob (F-statistic):             0.0902
Time:                        18:56:28   Log-Likelihood:                 19.198
No. Observations:                   8   AIC:                            -30.40
Df Residuals:                       4   BIC:                            -30.08
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.1272      0.078     -1.628      0.104      -0.280       0.026
PredTreatment              0.3358      0.147      2.283      0.022       0.047       0.624
Num_Date                   0.0474      0.019      2.524      0.012       0.011       0.084
PredTreatment:Num_Date    -0.1595      0.051     -3.142      0.002      -0.259      -0.060
==============================================================================
Omnibus:                        1.076   Durbin-Watson:                   2.254
Prob(Omnibus):                  0.584   Jarque-Bera (JB):                0.612
Skew:                           0.139   Prob(JB):                        0.736
Kurtosis:                       1.674   Cond. No.                         19.4
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [37]:
#It appear american groceries were more impacted than canadian groceries for tariffs, perhaps this impact was more measureable since Canada had higher variance changes
#It is also important to note that Canada has a lower population and are more likely to have higher variance as a result over time
#but American tariffs on American goods did significantly increase inflation in the post period and treatment period while Canadian tariffs appeared to lower the tariffs post period but increase tariffs during the treatment period signficant. Significance was not found for the two tariff periods in the Canadian groceries.

#The tariffs only caused a small blimp in the groceries while long term impact was harder to measure - let us investigate further



In [38]:
print(df_USA_CPI_groupby['Category'].unique())
USA_parallel_trends_groceries = plot_for_parallel_trends(df_USA_CPI_groupby, '2017-01-01', '2019-12-01', 'Groceries', 'Education & Reading', category_3 = None)

USA_parallel_trends_groceries

['Alcohol & Tobacco' 'Appliances & Equipment' 'Clothing & Footwear'
 'Communication' 'Education & Reading' 'Energy' 'Furniture & Home Decor'
 'Groceries' 'Health & Personal Care' 'Household Supplies'
 'Housing & Utilities' 'Housing, Rent, Tax & Insurance'
 'Recreation & Entertainment' 'Services' 'Transportation' 'Uncategorized'
 'Utilities' 'Vehicles']


/Users/richardwcheng/Desktop/GoodsComparison2018Tariffs/notebooks/../src/models/rc_differences.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [39]:
differences_differences(df_USA_CPI_Scaled, 'Groceries', 'Education & Reading', '2018-03-01', '2018-05-01', '2018-04-01')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     4.484
Date:                Thu, 20 Mar 2025   Prob (F-statistic):            0.00494
Time:                        18:56:28   Log-Likelihood:                -194.22
No. Observations:                 138   AIC:                             396.4
Df Residuals:                     134   BIC:                             408.2
Df Model:                           3                                         
Covariance Type:                  HC3                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.4096      0.081     -5.051      0.000      -0.569      -0.251
tariff_non_tariffed          0.4745      0.144      3.291      0.001       0.192       0.757
post                         0.1885      0.169      1.117      0.264      -0.142       0.519
tariff_non_tariffed:post    -0.1965      0.276     -0.711      0.477      -0.738       0.345
==============================================================================
Omnibus:                       22.452   Durbin-Watson:                   0.794
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.892
Skew:                           0.907   Prob(JB):                     3.23e-07
Kurtosis:                       4.382   Cond. No.                         11.9
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [40]:
differences_differences(df_USA_CPI_groupby, 'Groceries', 'Education & Reading', '2018-03-01', '2018-05-01', '2018-04-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                     2.259
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.322
Time:                        18:56:28   Log-Likelihood:                -2.5649
No. Observations:                   6   AIC:                             13.13
Df Residuals:                       2   BIC:                             12.30
Df Model:                           3                                         
Covariance Type:                  HC3                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -1.1668      0.533     -2.188      0.029      -2.212      -0.122
tariff_non_tariffed          2.2703      0.909      2.498      0.012       0.489       4.052
post                         1.2923      5.153      0.251      0.802      -8.807      11.391
tariff_non_tariffed:post    -2.3945      5.214     -0.459      0.646     -12.614       7.825
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.353
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.459
Skew:                          -0.000   Prob(JB):                        0.795
Kurtosis:                       1.646   Cond. No.                         6.32
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [41]:


#differences in differences appears to confirm little impact on overall groceries - let us examine why with datasets of export and import





In [42]:
USA_sales = pd.read_csv("../data/processed/USA_Sales_Processed.csv")
USA_sales_groceries_vehicles = pd.read_csv("../data/processed/USA_Sales_Groceries_Vehicles_Processed.csv")
print(USA_sales.tail())
print(USA_sales_groceries_vehicles.tail())

                    cat_idx                                    dt_idx  \
575623  Other Durable Goods         New Orders Percent Change Monthly   
575624  Other Durable Goods    Unfilled Orders Percent Change Monthly   
575625  Other Durable Goods    Unfilled Orders Percent Change Monthly   
575626  Other Durable Goods  Total Inventories Percent Change Monthly   
575627  Other Durable Goods  Total Inventories Percent Change Monthly   

        geo_idx  is_adj  val    REF_DATE  
575623        1       1 -0.5  2023-01-01  
575624        1       0  0.4  2023-01-01  
575625        1       1 -0.9  2023-01-01  
575626        1       0  2.0  2023-01-01  
575627        1       1  0.3  2023-01-01  
                                      cat_idx dt_idx  et_idx  geo_idx  is_adj  \
185555  72251X: Limited Service Eating Places    NaN       5        1       1   
185556  72251X: Limited Service Eating Places    NaN       5        1       1   
185557  72251X: Limited Service Eating Places    NaN       5

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_58411/834534902.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  USA_sales_groceries_vehicles = pd.read_csv("../data/processed/USA_Sales_Groceries_Vehicles_Processed.csv")


In [43]:
for column in USA_sales_groceries_vehicles['cat_idx'].unique():
    print(column)

44X72: Retail Trade and Food Services
44Y72: Retail Trade and Food Services, ex Auto
44Z72: Retail Trade and Food Services, ex Gas
44W72: Retail Trade and Food Services, ex Auto and Gas
44000: Retail Trade
4400A: Retail Trade, ex Auto
4400C: GAFO
441: Motor Vehicle and Parts Dealers
4411,4412: Auto and Other Motor Vehicles
4411: Automobile Dealers
44111: New Car Dealers
44112: Used Car Dealers
4413: Automotive Parts, Acc., and Tire Stores
4423X: Furniture, Home Furn, Electronics, and Appliance Stores
442: Furniture and Home Furnishings Stores
4421: Furniture Stores
4422: Home Furnishings Stores
44221: Floor Covering Stores
443: Electronics and Appliance Stores
443141: Household Appliance Stores
444: Building Mat. and Garden Equip. and Supplies Dealers
4441: Building Mat. and Supplies Dealers
44413: Hardware Stores
445: Food and Beverage Stores
4451: Grocery Stores
4453: Beer, Wine, and Liquor Stores
446: Health and Personal Care Stores
44611: Pharmacies and Drug Stores
447: Gasoline St

In [44]:
USA_sales['PrincipleStats'] = USA_sales['dt_idx']
USA_sales['GoodType'] = USA_sales['cat_idx']
USA_sales['VALUE'] = USA_sales["val"]



In [76]:
for column in USA_sales['PrincipleStats'].unique():
    print(column)

print(sep = '\n')
for column in USA_sales['GoodType'].unique():
    print(column)
print(sep = '\n')
for column in USA_sales.columns:
    print(column)

USA_sales['PrincipleStats'] = USA_sales["PrincipleStats"].astype(str)
USA_sales['GoodType'] = USA_sales['GoodType'].astype(str)

USA_sales.to_csv("../data/processed/USA_Sales_Processed_Final.csv", index=False)
df_sales.to_csv("../data/processed/Canada_Sales_Processed_Final.csv", index=False)

Value of Shipments
New Orders
Unfilled Orders
Total Inventories
Materials and Supplies Inventories
Work in Process Inventories
Finished Goods Inventories
Inventories to Shipments Ratios
Unfilled Orders to Shipments Ratios
Value of Shipments Percent Change Monthly
New Orders Percent Change Monthly
Unfilled Orders Percent Change Monthly
Total Inventories Percent Change Monthly
Materials & Supplies Inventories Percent Change Monthly
Work in Progress Inventories Percent Change Monthly
Finished Goods Inventories Percent Change Monthly

Total Manufacturing
Manufacturing Excluding Transportation
Manufacturing Excluding Defense
Manufacturing with Unfilled Orders
Durable Goods
Wood Products
Nonmetallic Mineral Products
Primary Metals
Iron and Steel Mills and Ferroalloy and Steel Product Manufacturing
Aluminum and Nonferrous Metal Products
Ferrous Metal Foundries
Fabricated Metal Products
Machinery
Farm Machinery and Equipment Manufacturing
Construction Machinery Manufacturing
Mining, Oil, and G

In [46]:
for column in USA_sales['GoodType'].unique():
    print(column)
    print(USA_sales[(USA_sales['GoodType']==column) & (USA_sales['PrincipleStats']=='Finished Goods Inventories Percent Change Monthly') & (USA_sales['REF_DATE']>='2017-01-01')&(USA_sales['REF_DATE']<='2020-02-01')])


USA_groceries_supply_demand = plot_supply_and_demand_canada(USA_sales, "Food Products", '2017-01-01', '2019-10-01', "Finished Goods Inventories Percent Change Monthly", "Value of Shipments")


Total Manufacturing
                    cat_idx  \
470138  Total Manufacturing   
470139  Total Manufacturing   
470140  Total Manufacturing   
470141  Total Manufacturing   
470142  Total Manufacturing   
...                     ...   
504961  Total Manufacturing   
522338  Total Manufacturing   
522339  Total Manufacturing   
522350  Total Manufacturing   
522351  Total Manufacturing   

                                                   dt_idx  geo_idx  is_adj  \
470138  Finished Goods Inventories Percent Change Monthly        1       0   
470139  Finished Goods Inventories Percent Change Monthly        1       0   
470140  Finished Goods Inventories Percent Change Monthly        1       0   
470141  Finished Goods Inventories Percent Change Monthly        1       0   
470142  Finished Goods Inventories Percent Change Monthly        1       0   
...                                                   ...      ...     ...   
504961  Finished Goods Inventories Percent Change Monthly    

In [47]:
#It seems that the tariffs increased finished inventory percentage change per month which decreased the following month meaning demand was changing based on the tariff changes in the news notably on 2018-04 and 2018-08 which indicates an instability in the market demand coinciding with momentary inflation for that month for American groceries.

In [48]:
#lets compare the change in variance of groceries in America to Canada to compare the effect of tariffs on groceries in both countries

mask_USA_groceries = df_USA_CPI_Scaled['Category'] == 'Groceries'
mask_Canada_groceries = df_Canada_CPI_Scaled['Category'] == 'Groceries'

df_USA_CPI_Scaled_groceries = df_USA_CPI_Scaled[mask_USA_groceries]
df_Canada_CPI_Scaled_groceries = df_Canada_CPI_Scaled[mask_Canada_groceries]

df_USA_CPI_Scaled_groceries['Country'] = ['USA']*len(df_USA_CPI_Scaled_groceries)
df_Canada_CPI_Scaled_groceries['Country'] = ['Canada']*len(df_Canada_CPI_Scaled_groceries)

df_USA_CPI_Scaled_groceries.drop(columns = ['Products and product groups'], inplace=True)
df_Canada_CPI_Scaled_groceries.drop(columns = ['Products and product groups'], inplace=True)

df_USA_CPI_Scaled_groc_var = df_USA_CPI_Scaled_groceries.groupby(['REF_DATE', 'Category', 'Country']).std().reset_index()
df_Canada_CPI_Scaled_groc_var = df_Canada_CPI_Scaled_groceries.groupby(['REF_DATE', 'Category', 'Country']).std().reset_index()

df_USA_Canada_groceries = pd.concat([df_USA_CPI_Scaled_groc_var, df_Canada_CPI_Scaled_groc_var])

variance_groceries_CPI = plot_structure(df_USA_Canada_groceries, 'Groceries', '2017-01-01', '2020-02-01', color = 'Country', x_label = "Date", y_label = "Variance of CPI index for Groceries", lines_to_plot = ['2017-08-01', '2017-11-01', '2019-05-01'], title = 'CPI Variance for Groceries in Canada and USA')

((USA_parallel_trends_groceries|Canada_parallel_trends_groceries)&(USA_groceries_supply_demand | Canada_groceries_supply_demand)).configure_axis(grid=False).properties(title = alt.TitleParams(text = 'US and Canada Groceries Inflation and Supply and Demand', subtitle = 'The tariffs caused a waves of inflation in groceries in both countries with food manufacturing being more affected in Canada than US')).save('US_Canada_Groceries_Inflation_Supply_Demand.html')


Product:  Groceries


/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_58411/129982412.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_USA_CPI_Scaled_groceries['Country'] = ['USA']*len(df_USA_CPI_Scaled_groceries)
/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_58411/129982412.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_groceries['Country'] = ['Canada']*len(df_Canada_CPI_Scaled_groceries)
/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_58411/129982412.py:12: Se

In [49]:
#It seems like the break down of the China talks decreased variance and the proceeding IP investigation increased variance in America first while Canada had a lag in their decrease.
#The IP investigation and the tariffs appeared to increase variance following the breakdown in talks meaning there was recooperation but America did not quite recover following the end of the tariffs in May 2019.
# the variance of the CPI decrease may mean that the consumer market relied on only essential grocery goods during the tariffs - we should further investigate 

In [50]:
df_Canada_Number_Groceries = pd.read_csv("../data/processed/Canada_Number_Groceries_Processed.csv")

In [51]:
print(df_Canada_Number_Groceries.head())
for uniq_product in df_Canada_Number_Groceries['North American Industry Classification System (NAICS)'].unique():
    print(uniq_product)

                     Sales  \
0       Total retail sales   
1  Retail e-commerce sales   
2       Total retail sales   
3       Total retail sales   
4       Total retail sales   

  North American Industry Classification System (NAICS)    REF_DATE  \
0                               Retail trade [44-45]     2017-01-01   
1                               Retail trade [44-45]     2017-01-01   
2              Motor vehicle and parts dealers [441]     2017-01-01   
3                          Automobile dealers [4411]     2017-01-01   
4                            New car dealers [44111]     2017-01-01   

        VALUE  
0  50417235.0  
1   1236885.0  
2  13561527.0  
3  11980440.0  
4  11020613.0  
Retail trade [44-45]
Motor vehicle and parts dealers [441]
Automobile dealers [4411]
New car dealers [44111]
Used car dealers [44112]
Other motor vehicle dealers [4412]
Automotive parts, accessories and tire retailers [4413]
Building material and garden equipment and supplies dealers [444]
Food 

In [52]:
lst_groceries_category = [0]*8 + [1]*6 + [0]*16
dict_groceries = {uniq_product: category for uniq_product, category in zip(df_Canada_Number_Groceries['North American Industry Classification System (NAICS)'].unique(), lst_groceries_category)}

In [53]:
df_Canada_Number_Groceries['Category'] = df_Canada_Number_Groceries['North American Industry Classification System (NAICS)'].map(dict_groceries)

In [54]:
df_Canada_Number_Groceries_groupby = df_Canada_Number_Groceries.copy()
df_Canada_Number_Groceries_groupby.drop(columns = ['North American Industry Classification System (NAICS)', 'Sales'], inplace=True)
df_Canada_Number_Groceries_groupby = df_Canada_Number_Groceries_groupby.groupby(['REF_DATE', 'Category']).std().reset_index()
df_Canada_Number_Groceries_groupby.reset_index(inplace=True)
mask = (df_Canada_Number_Groceries_groupby['Category']==1)
scaler = StandardScaler()
df_Canada_Number_Groceries_plot_all = df_Canada_Number_Groceries[df_Canada_Number_Groceries['Category']==1]
df_Canada_Number_Groceries_groupby.loc[mask, "VALUE"] = scaler.fit_transform(df_Canada_Number_Groceries_groupby.loc[mask, 'VALUE'].values.reshape(-1, 1))
plot_structure(df_Canada_Number_Groceries_groupby,1,'2017-01-01', '2020-02-01')
for unique_class in df_Canada_Number_Groceries_plot_all['North American Industry Classification System (NAICS)'].unique():
    scale = StandardScaler()
    mask = df_Canada_Number_Groceries_plot_all['North American Industry Classification System (NAICS)'] == unique_class
    df_Canada_Number_Groceries_plot_all.loc[mask, "VALUE"] = scale.fit_transform(df_Canada_Number_Groceries_plot_all[mask]['VALUE'].values.reshape(-1, 1))
alt.Chart(df_Canada_Number_Groceries_plot_all).mark_line().encode(
    x='REF_DATE',
    y='VALUE',
    color='North American Industry Classification System (NAICS)'
)

Product:  1


alt.Chart(...)

In [55]:
#It seems there was higher variance in terms of where the groceries were bought in Canada coinciding with results from the breakdown in talks with China, the IP announcment and the tariffs - there was a reliance on convenience retailers and vending machines during the tariff period which was then under-utilized durign the recovery period follow May 2019
# 
# This seems to indicate that the Canadian groceries for consumers did indeed recover following the tariffs and the recovery may have been stronger than American groceries.

#let us now turn out attention to clothing. 

In [56]:
plot_structure(df_Canada_CPI_groupby, 'Clothing & Footwear', '2017-01-01', '2019-10-01')

Product:  Clothing & Footwear


alt.Chart(...)

In [57]:
#It doesnt appear Canadian clothing and footwear was that impacted by tariffs but let us look closer

In [58]:
Canada_Supply_Demand_Clothing = plot_supply_and_demand_canada(df_sales, "Leather and allied product manufacturing [316]", "2017-01-01", "2019-10-01")

6624    112779.0
6625    118602.0
6626    116474.0
6627    113234.0
6628    111506.0
Name: VALUE, dtype: float64


In [59]:

Apparel_Canada = plot_supply_and_demand_canada(df_sales, "Apparel manufacturing [315]", "2017-01-01", "2019-10-01")

Apparel_Canada

6234    577572.0
6235    583848.0
6236    602944.0
6237    587686.0
6238    608707.0
Name: VALUE, dtype: float64


alt.Chart(...)

In [60]:
#IT seems like manufacturing demand for clothing took a hit during July which means there were less orders from clothing stores and supply for clothing was decreased

#Inflation should have increased a bit for clothing and footwear but it was the impact of trumps tariffs that caused the reduction in supply - this increase in inflation is difficult to measure in July due to the seasonal nature of clothing and footwear but perhaps we can look at regression discontinuity 

In [61]:
model_clothing_US_tariff_on_Canada, _, _, _ = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Clothing & Footwear', '2018-03-01', '2018-09-01', '2018-07-01', '2018-08-01')

model_clothing_US_tariff_on_Canada.summary()

Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     208.4
Date:                Thu, 20 Mar 2025   Prob (F-statistic):           5.71e-38
Time:                        18:56:33   Log-Likelihood:                -1.2430
No. Observations:                 112   AIC:                             8.486
Df Residuals:                     109   BIC:                             16.64
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6762      0.036     18.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.027
Method:                 Least Squares   F-statistic:                   0.02333
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.995
Time:                        18:56:33   Log-Likelihood:                -149.79
No. Observations:                 112   AIC:                             307.6
Df Residuals:                     108   BIC:                             318.4
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0167      0.107     -0.157      0.876      -0.226       0.193
PredTreatment              0.0221      0.230      0.096      0.924      -0.429       0.473
Num_Date                  -0.0019      0.043     -0.046      0.964      -0.085       0.082
PredTreatment:Num_Date     0.0412      0.190      0.217      0.828      -0.331       0.414
==============================================================================
Omnibus:                        2.528   Durbin-Watson:                   2.934
Prob(Omnibus):                  0.283   Jarque-Bera (JB):                2.024
Skew:                           0.197   Prob(JB):                        0.364
Kurtosis:                       3.528   Cond. No.                         7.97
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [62]:
model_clothing_US_tariff_groupby, clothing_chart_clothing_US_original, clothing_chart_clothing_US_diff, clothing_chart_clothing_US_vtrend, clothing_chart_clothing_US_trend = regression_discontinuity_model(df_Canada_CPI_groupby, 'Clothing & Footwear', '2017-08-01', '2019-10-01', '2019-03-01', '2019-08-01', seasonality=True)

model_clothing_US_tariff_groupby.summary()

Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.558
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     13.19
Date:                Thu, 20 Mar 2025   Prob (F-statistic):           0.000137
Time:                        18:56:33   Log-Likelihood:                -3.5854
No. Observations:                  27   AIC:                             13.17
Df Residuals:                      24   BIC:                             17.06
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5468      0.108      5.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            VALUE_TREND   R-squared:                       0.588
Model:                            OLS   Adj. R-squared:                  0.534
Method:                 Least Squares   F-statistic:                     8.584
Date:                Thu, 20 Mar 2025   Prob (F-statistic):           0.000526
Time:                        18:56:33   Log-Likelihood:                 70.992
No. Observations:                  27   AIC:                            -134.0
Df Residuals:                      23   BIC:                            -128.8
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.0412      0.005      8.960      0.000       0.032       0.050
PredTreatment              0.0263      0.015      1.769      0.077      -0.003       0.055
Num_Date                   0.0013      0.000      3.152      0.002       0.001       0.002
PredTreatment:Num_Date     0.0080      0.002      3.657      0.000       0.004       0.012
==============================================================================
Omnibus:                        1.082   Durbin-Watson:                   1.067
Prob(Omnibus):                  0.582   Jarque-Bera (JB):                0.832
Skew:                          -0.033   Prob(JB):                        0.660
Kurtosis:                       2.143   Cond. No.                         55.3
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [63]:
#this visual trend is not picked up by statistics - which means there was lower impact on inflation perhaps because of the already abundance of clothing in stores and the already lack of demand

#In fact the effect of the tariffs was not seen until during May 2019 - August 2019 which coincides with food manufacturing increases meaning there was a lack of supply for clothing which is why there was inflation, this impact was significant. 

In [64]:
#It appears to have caused a decrease in orders or manufacturing demand which means less supply for clothing and footwear, however, demand was still above supply in this time frame - the impact may not have been felt until in May 2019 because of the already high supply of clothing and footwear in retail stores



In [65]:
for column in USA_sales['GoodType'].unique():
    print(column)
    print(USA_sales[(USA_sales['GoodType']==column) & (USA_sales['PrincipleStats']=='Finished Goods Inventories Percent Change Monthly') & (USA_sales['REF_DATE']>='2017-01-01')&(USA_sales['REF_DATE']<='2020-02-01')])


USA_leather = plot_supply_and_demand_canada(USA_sales, "Leather and Allied Products", '2017-01-01', '2019-10-01', "Finished Goods Inventories Percent Change Monthly", "Value of Shipments")


Total Manufacturing
                    cat_idx  \
470138  Total Manufacturing   
470139  Total Manufacturing   
470140  Total Manufacturing   
470141  Total Manufacturing   
470142  Total Manufacturing   
...                     ...   
504961  Total Manufacturing   
522338  Total Manufacturing   
522339  Total Manufacturing   
522350  Total Manufacturing   
522351  Total Manufacturing   

                                                   dt_idx  geo_idx  is_adj  \
470138  Finished Goods Inventories Percent Change Monthly        1       0   
470139  Finished Goods Inventories Percent Change Monthly        1       0   
470140  Finished Goods Inventories Percent Change Monthly        1       0   
470141  Finished Goods Inventories Percent Change Monthly        1       0   
470142  Finished Goods Inventories Percent Change Monthly        1       0   
...                                                   ...      ...     ...   
504961  Finished Goods Inventories Percent Change Monthly    

In [66]:
#let us look at the American dataset to see if the same trend is seen

plot_structure(df_USA_CPI_groupby, 'Clothing & Footwear', '2017-01-01', '2019-10-01')

Product:  Clothing & Footwear


alt.Chart(...)

In [67]:
model_clothing_US_tariff, USA_chart_clothing_original, US_chart_clothing_diff, US_chart_clothing_vtrend, US_chart_clothing_trend = regression_discontinuity_model(df_USA_CPI_groupby, 'Clothing & Footwear', '2017-01-01', '2019-08-01', '2017-10-01', '2019-02-01', seasonality=True)

print(model_clothing_US_tariff.summary())
model_clothing_US_tariff, USA_chart_clothing_original, US_chart_clothing_diff, US_chart_clothing_vtrend, US_chart_clothing_trend = regression_discontinuity_model(df_USA_CPI_groupby, 'Clothing & Footwear', '2017-01-01', '2019-08-01', '2017-10-01', '2019-02-01', seasonality=True, fuzzy_sharp_omit = True)

model_clothing_US_tariff.summary()


Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     93.36
Date:                Thu, 20 Mar 2025   Prob (F-statistic):           4.11e-13
Time:                        18:56:37   Log-Likelihood:               -0.95444
No. Observations:                  31   AIC:                             7.909
Df Residuals:                      28   BIC:                             12.21
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.4735      0.071      6.

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            VALUE_TREND   R-squared:                       0.621
Model:                            OLS   Adj. R-squared:                  0.526
Method:                 Least Squares   F-statistic:                     10.91
Date:                Thu, 20 Mar 2025   Prob (F-statistic):           0.000959
Time:                        18:56:37   Log-Likelihood:                 53.715
No. Observations:                  16   AIC:                            -99.43
Df Residuals:                      12   BIC:                            -96.34
Df Model:                           3                                         
Covariance Type:                  HC3                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -0.0197      0.009     -2.197      0.028      -0.037      -0.002
above_or_below              0.0502      0.050      1.004      0.316      -0.048       0.148
Num_Date                 7.154e-05      0.002      0.038      0.970      -0.004       0.004
above_or_below:Num_Date    -0.0015      0.003     -0.503      0.615      -0.007       0.004
==============================================================================
Omnibus:                       22.668   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.624
Skew:                           2.138   Prob(JB):                     1.65e-06
Kurtosis:                       7.653   Cond. No.                         327.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [68]:
#It seems that both trumps tariffs caused American goods to increase in inflation while Canadian/Chinese tariffs caused a decrease in inflation for American clothings and footwear

#let us investigate further with regression discontinuity

model_clothing_US_tariff, _, _, _ = regression_discontinuity_model(df_USA_CPI_groupby, 'Clothing & Footwear', '2017-10-01', '2018-06-01', '2018-02-01', '2018-04-01')

model_clothing_US_tariff.summary()

Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     13.29
Date:                Thu, 20 Mar 2025   Prob (F-statistic):            0.00625
Time:                        18:56:37   Log-Likelihood:                0.69908
No. Observations:                   9   AIC:                             4.602
Df Residuals:                       6   BIC:                             5.194
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6391      0.123      5.

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)
/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                    0.5992
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.643
Time:                        18:56:37   Log-Likelihood:                 5.2059
No. Observations:                   9   AIC:                            -2.412
Df Residuals:                       5   BIC:                            -1.623
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0232      0.087     -0.266      0.790      -0.194       0.148
PredTreatment              0.2397      0.212      1.130      0.258      -0.176       0.655
Num_Date                  -0.0040      0.027     -0.145      0.885      -0.058       0.050
PredTreatment:Num_Date     0.0198      0.141      0.140      0.889      -0.257       0.297
==============================================================================
Omnibus:                        1.761   Durbin-Watson:                   2.815
Prob(Omnibus):                  0.415   Jarque-Bera (JB):                1.088
Skew:                          -0.615   Prob(JB):                        0.580
Kurtosis:                       1.821   Cond. No.                         8.22
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [69]:
model_clothing_US_tariff, _, _, _ = regression_discontinuity_model(df_USA_CPI_Scaled, 'Clothing & Footwear', '2017-10-01', '2018-06-01', '2018-02-01', '2018-04-01')

model_clothing_US_tariff.summary()

Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     530.7
Date:                Thu, 20 Mar 2025   Prob (F-statistic):           9.10e-69
Time:                        18:56:37   Log-Likelihood:                 11.884
No. Observations:                 153   AIC:                            -17.77
Df Residuals:                     150   BIC:                            -8.678
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6391      0.024     26.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          VALUE_DETREND   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                   0.01800
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.997
Time:                        18:56:37   Log-Likelihood:                -178.21
No. Observations:                 153   AIC:                             364.4
Df Residuals:                     149   BIC:                             376.5
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.0153      0.083      0.184      0.854      -0.147       0.177
PredTreatment             -0.0226      0.203     -0.111      0.911      -0.421       0.376
Num_Date                   0.0003      0.026      0.013      0.990      -0.051       0.051
PredTreatment:Num_Date    -0.0057      0.098     -0.058      0.954      -0.199       0.187
==============================================================================
Omnibus:                        6.695   Durbin-Watson:                   2.957
Prob(Omnibus):                  0.035   Jarque-Bera (JB):                6.991
Skew:                          -0.360   Prob(JB):                       0.0303
Kurtosis:                       3.760   Cond. No.                         8.22
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [70]:
mask_USA_clothing = df_USA_CPI_Scaled['Category'] == 'Clothing & Footwear'
mask_Canada_clothing = df_Canada_CPI_Scaled['Category'] == 'Clothing & Footwear'

df_USA_CPI_Scaled_clothing = df_USA_CPI_Scaled[mask_USA_clothing]
df_Canada_CPI_Scaled_clothing = df_Canada_CPI_Scaled[mask_Canada_clothing]
print(df_USA_CPI_Scaled_clothing.head())
df_USA_CPI_Scaled_clothing['Country'] = ['USA']*len(df_USA_CPI_Scaled_clothing)
df_Canada_CPI_Scaled_clothing['Country'] = ['Canada']*len(df_Canada_CPI_Scaled_clothing)

df_USA_CPI_Scaled_clothing.drop(columns = ['Products and product groups'], inplace=True)
df_Canada_CPI_Scaled_clothing.drop(columns = ['Products and product groups'], inplace=True)

df_USA_CPI_Scaled_clothing_var = df_USA_CPI_Scaled_clothing.groupby(['REF_DATE', 'Category', 'Country']).std().reset_index()
df_Canada_CPI_Scaled_clothing_var = df_Canada_CPI_Scaled_clothing.groupby(['REF_DATE', 'Category', 'Country']).std().reset_index()

df_USA_Canada_clothing = pd.concat([df_USA_CPI_Scaled_clothing_var, df_Canada_CPI_Scaled_clothing_var])
print(df_USA_Canada_clothing.head())
CPI_variance_clothing = plot_structure(df_USA_Canada_clothing, 'Clothing & Footwear', '2017-01-01', '2020-02-01', color = 'Country', lines_to_plot = ['2019-05-01', '2019-11-01'], x_label = "Date", y_label = "Variance of CPI index for Clothing & Footwear", title = 'CPI Variance for Clothing & Footwear in Canada and USA')
(variance_groceries_CPI & CPI_variance_clothing).configure_axis(grid=False)

       REF_DATE Products and product groups     VALUE             Category
576  2017-01-01                     Apparel  0.663325  Clothing & Footwear
577  2017-02-01                     Apparel  0.686137  Clothing & Footwear
578  2017-03-01                     Apparel  0.877558  Clothing & Footwear
579  2017-04-01                     Apparel  0.802832  Clothing & Footwear
580  2017-05-01                     Apparel  0.573098  Clothing & Footwear
     REF_DATE             Category Country     VALUE
0  2017-01-01  Clothing & Footwear     USA  0.649403
1  2017-02-01  Clothing & Footwear     USA  0.755559
2  2017-03-01  Clothing & Footwear     USA  0.749317
3  2017-04-01  Clothing & Footwear     USA  0.715454
4  2017-05-01  Clothing & Footwear     USA  0.801528
Product:  Clothing & Footwear


/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_58411/833098296.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_USA_CPI_Scaled_clothing['Country'] = ['USA']*len(df_USA_CPI_Scaled_clothing)
/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_58411/833098296.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_clothing['Country'] = ['Canada']*len(df_Canada_CPI_Scaled_clothing)
/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_58411/833098296.py:10: Setting

alt.VConcatChart(...)

In [71]:
((US_chart_clothing_trend | clothing_chart_clothing_US_trend) & (USA_leather | Apparel_Canada)).properties(title = alt.TitleParams(text = 'Clothing & Footwear Inflation and Supply and Demand', subtitle = 'The tariffs impact caused a rise in inflation which, in the post period seemed to affect Canada more than America, particularly in regards to manufacturing')).save('US_Canada_Clothing_Inflation_Supply_Demand.html')
display(((US_chart_clothing_trend | clothing_chart_clothing_US_trend) & (USA_leather | Apparel_Canada)).configure_axis(grid=False).properties(title = alt.TitleParams(text = 'Clothing & Footwear Inflation and Supply and Demand', subtitle = 'The tariffs impact caused a rise in inflation which, in the post period seemed to affect Canada more than America, particularly in regards to manufacturing')))

alt.VConcatChart(...)

In [72]:
#It seems clothing and footwear have a seasonal impact for both countries but
#The american market seemed less impacted by the tariffs and while clothing did appear to reduce supply, the abundance of normal clothing perhaps was able to account for it. In fact it appears manufacturing was not completely impacted except for month to month during tariffs until near COVID period.
    
#The Canadian market was more impacted by the tariffs and the recovery was not as strong as the American market - the impact of the tariffs from Trump began seeing an effect in March 2019 due to the high supply of clothing already in retail stores. 
#Manufacturing for clothing was affected by August 2018 but impact on retail stores was not until March 2019 and was very disrupted near COVID period.




In [73]:
#This ends the groceries and clothing/footwear section of our analysis.